In [1]:
import torch

from diffdrr import Detector

In [2]:
Aorig = torch.arange(16).reshape(4, 4)
print(Aorig)

A = torch.flip(Aorig, [0])

spacing = torch.tensor([1., 1.])
isocenter = torch.tensor([0., 0.])
dims = torch.tensor(A.shape) + 1

nx, ny = dims
dx, dy = spacing

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])


In [3]:
source = torch.tensor([-1., 4.5], requires_grad=True)
target = torch.tensor([4.5, -0.5], requires_grad=True)

sx, sy = source

In [4]:
alphax = torch.arange(nx, dtype=torch.float32) * dx - sx
alphay = torch.arange(ny, dtype=torch.float32) * dy - sy

sdd = target - source
alphax = alphax / sdd[0]
alphay = alphay / sdd[1]

In [5]:
alphax, alphay

(tensor([0.1818, 0.3636, 0.5455, 0.7273, 0.9091], grad_fn=<DivBackward0>),
 tensor([0.9000, 0.7000, 0.5000, 0.3000, 0.1000], grad_fn=<DivBackward0>))

In [6]:
planes = torch.zeros(2)
alpha0 = (isocenter + planes * spacing - source) / sdd
planes = dims - 1
alpha1 = (isocenter + planes * spacing - source) / sdd
alphas = torch.stack([alpha0, alpha1])

alphamin = alphas.min(dim=0).values.max(dim=-1).values
alphamax = alphas.max(dim=0).values.min(dim=-1).values

alphamin, alphamax

(tensor(0.1818, grad_fn=<MaxBackward0>),
 tensor(0.9000, grad_fn=<MinBackward0>))

In [7]:
alphas = torch.hstack([alphax, alphay])
alphas

tensor([0.1818, 0.3636, 0.5455, 0.7273, 0.9091, 0.9000, 0.7000, 0.5000, 0.3000,
        0.1000], grad_fn=<CatBackward0>)

In [8]:
good_idxs = torch.logical_and(alphas >= alphamin, alphas <= alphamax)
alphas[~good_idxs] = torch.nan
alphas = alphas.sort(dim=0).values
alphas

tensor([0.1818, 0.3000, 0.3636, 0.5000, 0.5455, 0.7000, 0.7273, 0.9000,    nan,
           nan], grad_fn=<SortBackward0>)

In [9]:
alphamids = (alphas[0:-1] + alphas[1:]) / 2
alphamids

tensor([0.2409, 0.3318, 0.4318, 0.5227, 0.6227, 0.7136, 0.8136,    nan,    nan],
       grad_fn=<DivBackward0>)

In [29]:
idxs = (source + alphamids.unsqueeze(-1) * sdd - isocenter) / spacing
idxs = idxs.trunc()
idxs
# idxs = idxs[:, 0] + idxs[:, 1]

tensor([[0., 3.],
        [0., 2.],
        [1., 2.],
        [1., 1.],
        [2., 1.],
        [2., 0.],
        [3., 0.],
        [nan, nan],
        [nan, nan]], grad_fn=<TruncBackward0>)

In [30]:
idxs = (idxs[:, 0] + idxs[:, 1] * (ny-1)).long()
idxs[idxs < 0] = 0
idxs

tensor([12,  8,  9,  5,  6,  2,  3,  0,  0])

In [31]:
torch.take(A, idxs)

tensor([ 0,  4,  5,  9, 10, 14, 15, 12, 12])

In [32]:
Aorig

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [37]:
step_length = torch.diff(alphas)
voxel_value = torch.take(A, idxs)
torch.nansum(step_length * voxel_value)

tensor(5.8636, grad_fn=<NansumBackward0>)